In [18]:
import sqlite3
import seaborn
import pandas as pd

def my_row_factory(cur, row):
    d={}
    for idx, col in enumerate(cur.description):
        d[col[0].lower()] = row[idx]
    return d

conn = sqlite3.connect("../data_aggregation/options_database.db")
conn.row_factory = my_row_factory
cursor = conn.cursor()


In [46]:
query = """
select 
    contract_data.*,
    price_data.close as stock_price
from contract_data
join price_data
on (price_data.Symbol = contract_data.Symbol) and (price_data.date_of_close = contract_data.obs_date)
    where
        contract_data.symbol ='SPY'
        and obs_date = '2023-05-12'
;
"""
cursor.execute(query);

data = cursor.fetchall()

test = data[0]

In [28]:
from datetime import datetime

In [36]:
T = datetime.strptime(test['exp_date'], '%Y-%m-%d') - datetime.strptime(test['obs_date'], '%Y-%m-%d')

In [44]:
T.days

28

In [49]:
df = pd.DataFrame(data, columns=['symbol', 'contract_type', 'strike', 'bid', 'midpoint', 'ask', 'last', 'volume', 'open_int', 'obs_date', 'exp_date', 'stock_price'])
df.head()

,symbol,contract_type,strike,bid,midpoint,ask,last,volume,open_int,obs_date,exp_date,stock_price
0,SPY,call,275.0,137.02,137.34,137.66,0.0,0,0.0,2023-05-12,2023-06-09,411.59
1,SPY,call,280.0,132.04,132.36,132.68,0.0,0,0.0,2023-05-12,2023-06-09,411.59
2,SPY,call,285.0,127.07,127.38,127.70,0.0,0,0.0,2023-05-12,2023-06-09,411.59
3,SPY,call,290.0,122.10,122.42,122.73,0.0,0,0.0,2023-05-12,2023-06-09,411.59
4,SPY,call,295.0,117.13,117.44,117.76,0.0,0,0.0,2023-05-12,2023-06-09,411.59


In [31]:
display(df.obs_date.value_counts().head(4))
ddf = df[df.obs_date == '2023-05-12'].copy()
ddf =ddf.sort_values(by=['strike', 'contract_type'])

2023-05-12    618
2023-05-19    614
2023-06-16    570
2023-05-26    504
Name: obs_date, dtype: int64

In [35]:
def out_of_money(x):
    if x.contract_type == 'call':
        if(x.strike > x.stock_price):
            return True
    else:
        if(x.strike < x.stock_price):
            return True
    return False

ddf[ddf[["contract_type", "strike", "stock_price"]].apply(out_of_money, axis=1)]

,symbol,contract_type,strike,bid,midpoint,ask,last,volume,open_int,obs_date,exp_date,stock_price
922,SPY,put,150.0,0.00,0.00,0.01,0.01,159,64782.0,2023-05-12,2023-06-16,411.59
923,SPY,put,160.0,0.00,0.00,0.01,0.01,1,17267.0,2023-05-12,2023-06-16,411.59
924,SPY,put,170.0,0.01,0.02,0.02,0.01,0,13384.0,2023-05-12,2023-06-16,411.59
925,SPY,put,180.0,0.01,0.02,0.02,0.01,1,9169.0,2023-05-12,2023-06-16,411.59
719,SPY,put,185.0,0.01,0.02,0.02,0.01,0,5329.0,2023-05-12,2023-06-16,411.59
...,...,...,...,...,...,...,...,...,...,...,...,...
714,SPY,call,695.0,0.00,0.00,0.01,0.01,0,1440.0,2023-05-12,2023-06-16,411.59
715,SPY,call,700.0,0.00,0.00,0.01,0.01,0,1081.0,2023-05-12,2023-06-16,411.59
716,SPY,call,705.0,0.00,0.00,0.01,0.01,0,1228.0,2023-05-12,2023-06-16,411.59
717,SPY,call,710.0,0.00,0.00,0.01,0.01,0,141.0,2023-05-12,2023-06-16,411.59


In [36]:
ddf['exp_date'].unique()

array(['2023-06-16', '2023-06-09'], dtype=object)

## NO PANDAS CALCS

In [137]:
def my_row_factory(cur, row):
    d={}
    for idx, col in enumerate(cur.description):
        d[col[0].lower()] = row[idx]
    return d

conn = sqlite3.connect("../data_aggregation/options_database.db")
conn.row_factory = my_row_factory
cursor = conn.cursor()

query = """
select 
    contract_data.*,
    price_data.close as stock_price
from contract_data
join price_data
on (price_data.Symbol = contract_data.Symbol) and (price_data.date_of_close = contract_data.obs_date)
    where
        contract_data.symbol ='SPY'
        and obs_date = '2022-05-20'
;
"""
# '2023-05-12'


cursor.execute(query);
data = cursor.fetchall()
# conn.close()

In [138]:
len(data)

1736

In [135]:
%%time

dt = filter(out_of_money, data)

calls = filter(lambda x: x['contract_type']=='call', dt)
puts = filter(lambda x: x['contract_type']=='put', dt)

calls = sorted(calls, key=lambda x: abs(x['strike'] - x['stock_price']))
puts = sorted(puts, key=lambda x: abs(x['strike'] - x['stock_price']))

calls2 = filter_serious(calls)
puts2 = filter_serious(puts)

CPU times: total: 0 ns
Wall time: 2 ms


In [139]:
%%time
data = [opt for opt in data if out_of_money(opt)]

calls = [opt for opt in data if (opt['contract_type'] == 'call')]
puts = [opt for opt in data if (opt['contract_type'] == 'put')]

calls = sorted(calls, key=lambda x: abs(x['strike'] - x['stock_price']))
puts = sorted(puts, key=lambda x: abs(x['strike'] - x['stock_price']))

calls2 = filter_serious(calls)
puts2 = filter_serious(puts)

CPU times: total: 0 ns
Wall time: 2 ms


In [140]:
len(puts2)

345

In [84]:
def out_of_money(x):
    if x['contract_type'] == 'call':
        if(x['strike'] > x['stock_price']):
            return True
    else:
        if(x['strike'] < x['stock_price']):
            return True
    return False

def check_serious(bid, ask):
    return (bid > 0 and ask > 0)

def filter_serious(ls):
    ## Takes in a list of options sorted by ascending difference between strike and stock_price.
    new_ls = []
    prev_serious = True
    for opt in ls:
        if check_serious(opt['bid'], opt['ask']):
            new_ls.append(opt)
            prev_serious = True
        else:
            if not prev_serious:
                break
            prev_serious = False
            
    return new_ls
            

## Calculating the Risk Free Rate

In [7]:

QUERY = """
Select 
    *
from treasury_rates
where date_of = '2023-04-03'
"""
cursor.execute(QUERY)
data = cursor.fetchone()
data

{'date_of': '2023-04-03',
 '1mo': 4.7,
 '2mo': 4.79,
 '3mo': 4.9,
 '4mo': 4.98,
 '6mo': 4.88,
 '1yr': 4.6,
 '2yr': 3.97,
 '3yr': 3.73,
 '5yr': 3.52,
 '7yr': 3.48,
 '10yr': 3.43,
 '20yr': 3.78,
 '30yr': 3.64}

In [10]:
test = data

In [12]:
# PERIODS = {'1mo': 1/12, '2mo': 2/12, '3mo':3/12, '4mo':4/12, '6mo':1/2, '1yr':1, '2yr':2, '3yr':3, '5yr':5, '7yr':7, '10yr':10, '20yr':20, '30yr':30}
PERIODS = {'1mo': 30/365, '2mo': 60/365, '3mo':91/365, '4mo':121/365, '6mo':182/365, '1yr':1, '2yr':2, '3yr':3, '5yr':5, '7yr':7, '10yr':10, '20yr':20, '30yr':30}
# PERIODS = {'1mo': 30, '2mo': 60, '3mo':91, '4mo':121, '6mo':182, '1yr':365, '2yr':365*2, '3yr':365*3, '5yr':365*5, '7yr':365*7, '10yr':365*10, '20yr':365*20, '30yr':365*30}

from scipy.interpolate import CubicSpline
x = [PERIODS[i] for i in list(test.keys())[1:]]
y = [j for j in list(test.values())[1:]]


cs = CubicSpline(x, y)

print(cs((24/365)))

4.686279387561978


In [75]:
y = [0.03, 0.02, 0.04, 0.05, 0.08, 0.11, 0.22, 0.59, 1, 1.37, 2.03, 2.21]
x.remove(PERIODS['4mo'])

In [106]:
cs2 = CubicSpline(x, y)
r1 = cs2((31))
print(r1)

0.028719427541638892


In [104]:
r1 = cs2(28.786)
print(r1)

0.03166565052775329


In [72]:
x

[0.0821917808219178,
 0.1643835616438356,
 0.2493150684931507,
 0.4986301369863014,
 1,
 2,
 3,
 5,
 7,
 10,
 20,
 30]

In [61]:
y

[0.03, 0.02, 0.04, 0.05, 0.08, 0.11, 0.22, 0.59, 1, 1.37, 2.03, 2.21]

In [62]:
x

[0.08333333333333333,
 0.16666666666666666,
 0.3333333333333333,
 0.5,
 1,
 2,
 3,
 5,
 7,
 10,
 20,
 30]

In [8]:
import vix_calc

In [9]:
vix_calc.get_rfr('2023-04-03', (24/365))

array(4.68627939)